# Exploratory Data Analysis (EDA) of the MovieLens Dataset

This notebook is an exploratory data analysis (EDA) of the MovieLens dataset. The ratings are integers between 1 and 5. The dataset also contains the genre of each movie.

Download the dataset:
```python
poetry run python download_dataset.py
```


In [1]:
import pandas as pd

In [6]:
ratings = pd.read_csv("data/extracted/ratings.csv")
movie_meta = pd.read_csv("data/extracted/movies.csv")

# Convert timestamp to datetime in new column
ratings["datetime"] = pd.to_datetime(ratings["timestamp"], unit="s")

In [10]:
# Show basic dataset info
ratings.info()
display(ratings.describe())
display(ratings.head(2))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33832162 entries, 0 to 33832161
Data columns (total 5 columns):
 #   Column     Dtype         
---  ------     -----         
 0   userId     int64         
 1   movieId    int64         
 2   rating     float64       
 3   timestamp  int64         
 4   datetime   datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 1.3 GB


,userId,movieId,rating,timestamp,datetime
count,3.383216e+07,3.383216e+07,3.383216e+07,3.383216e+07,33832162
mean,1.654380e+05,2.831348e+04,3.542540e+00,1.269362e+09,2010-03-23 16:27:56.735719936
min,1.000000e+00,1.000000e+00,5.000000e-01,7.896520e+08,1995-01-09 11:46:44
25%,8.295300e+04,1.219000e+03,3.000000e+00,1.046718e+09,2003-03-03 18:56:32
50%,1.661290e+05,3.263000e+03,4.000000e+00,1.264740e+09,2010-01-29 04:42:24
75%,2.474500e+05,4.049100e+04,4.000000e+00,1.496919e+09,2017-06-08 10:56:19.249999872
max,3.309750e+05,2.889830e+05,5.000000e+00,1.689843e+09,2023-07-20 08:53:33
std,9.534122e+04,4.992865e+04,1.063959e+00,2.541023e+08,NaN


,userId,movieId,rating,timestamp,datetime
0,1,1,4.0,1225734739,2008-11-03 17:52:19
1,1,110,4.0,1225865086,2008-11-05 06:04:46


In [16]:
ratings[ratings["datetime"] > "2022-01-01"]["rating"].value_counts().sort_index()

rating
0.5     36249
1.0     33747
1.5     33810
2.0     79071
2.5    106201
3.0    228964
3.5    268935
4.0    345516
4.5    178229
5.0    205249
Name: count, dtype: int64

In [3]:
# Sample 10 users for testing
user_sample = ratings.userId.sample(10, random_state=42)
ratings = ratings[ratings.userId.isin(user_sample)]

In [4]:
# Drop movies never rated
movie_meta = movie_meta[movie_meta["movieId"].isin(ratings["movieId"])]

# Rename columns
column_rename = {"movieId": "movie_id", "userId": "user_id"}
ratings = ratings.rename(columns=column_rename)
movie_meta = movie_meta.rename(columns=column_rename)



In [5]:
def genre_features(df: pd.DataFrame) -> pd.DataFrame:
    """Extracts dummy features from movie dataframe `genres` column.

    Returns a new dataframe with the original `genres` column removed and
    replaced with dummy columns for each genre.
    """
    dummies = (
        df["genres"]
        .str.get_dummies(sep="|")
        .drop(columns="(no genres listed)")
        .rename(columns=lambda x: x.lower().replace("-", "_"))
    )
    df.drop(columns=["genres"], inplace=True)
    return pd.concat([df, dummies], axis=1)


def split_title_year(df: pd.DataFrame) -> pd.DataFrame:
    """Extracts year from title and creates a new column for it.

    Returns a new dataframe with the year removed from the original `title`
    column and a new `year` column added.

    The year column will have null values for movies where the year could
    not be extracted from the title be of type `float`.
    """
    year_pattern = r"\((\d{4})\)"
    df["year"] = df.title.str.extract(year_pattern)
    df["title"] = df.title.str.replace(year_pattern, "", regex=True).str.strip()
    return df


def impute_missing_year(
    df_movies: pd.DataFrame, df_ratings: pd.DataFrame
) -> pd.DataFrame:
    """Imputes missing years in the `year` column.

    Returns a new dataframe with missing years filled in with the
    earliest year the movie was rated.
    """
    missing_mask = df_movies["year"].isna()
    year_first_rated = (
        df_ratings.sort_values("datetime")
        .drop_duplicates("movie_id", keep="first")
        .set_index("movie_id")["datetime"]
        .apply(lambda x: x.year)
    )
    df_movies.loc[missing_mask, "year"] = df_movies.loc[missing_mask, "movie_id"].map(
        year_first_rated
    )

    assert df_movies["year"].isna().sum() == 0
    df_movies["year"] = df_movies.year.astype(int)
    return df_movies

In [6]:
def prepare_movie_data(
    df_movies: pd.DataFrame, df_ratings: pd.DataFrame
) -> pd.DataFrame:
    """Prepares the movie data for use in a recommender system."""
    df_movies = genre_features(df_movies)
    df_movies = split_title_year(df_movies)
    df_movies = impute_missing_year(df_movies, df_ratings)
    return df_movies

In [7]:
movie_features = prepare_movie_data(movie_meta, ratings)

display(movie_features.info())
display(movie_features.head())

<class 'pandas.core.frame.DataFrame'>
Index: 3097 entries, 0 to 86492
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   movie_id     3097 non-null   int64 
 1   title        3097 non-null   object
 2   action       3097 non-null   int64 
 3   adventure    3097 non-null   int64 
 4   animation    3097 non-null   int64 
 5   children     3097 non-null   int64 
 6   comedy       3097 non-null   int64 
 7   crime        3097 non-null   int64 
 8   documentary  3097 non-null   int64 
 9   drama        3097 non-null   int64 
 10  fantasy      3097 non-null   int64 
 11  film_noir    3097 non-null   int64 
 12  horror       3097 non-null   int64 
 13  imax         3097 non-null   int64 
 14  musical      3097 non-null   int64 
 15  mystery      3097 non-null   int64 
 16  romance      3097 non-null   int64 
 17  sci_fi       3097 non-null   int64 
 18  thriller     3097 non-null   int64 
 19  war          3097 non-null   in

None

,movie_id,title,action,adventure,animation,children,comedy,crime,documentary,drama,...,horror,imax,musical,mystery,romance,sci_fi,thriller,war,western,year
0,1,Toy Story,0,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1995
1,2,Jumanji,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1995
2,3,Grumpier Old Men,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,1995
5,6,Heat,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,1995
6,7,Sabrina,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,1995


In [8]:
from pandas.core.groupby.generic import DataFrameGroupBy

df_example_user_data = pd.DataFrame(
    {
        "user_id": [1, 1, 1, 1, 2, 2],
        "movie_id": [1, 2, 3, 4, 3, 1],
        "rating": [5, 4, 3, 2, 4, 4],
        "datetime": [
            pd.Timestamp("2021-01-01 10:00:00"),
            pd.Timestamp("2021-01-20 13:00:00"),
            pd.Timestamp("2021-02-05 15:00:00"),
            pd.Timestamp("2021-02-10 09:00:00"),
            pd.Timestamp("2021-01-05 10:00:00"),
            pd.Timestamp("2021-03-02 10:00:00"),
        ],
    }
)


# Create user point-in-time features
def user_num_ratings_last_30d(df: pd.DataFrame) -> pd.DataFrame:
    """Calculates the number of ratings a user has made in the past 30 days.

    Returns a sparse dataframe with the number of ratings a user has made in
    the past 30 days at each point in time.

    Example:
    user_id  movie_id  rating   datetime
    1        1         5        2021-01-01 10:00:00
    1        2         4        2021-01-20 13:00:00
    1        3         3        2021-02-05 15:00:00
    1        4         2        2021-02-10 09:00:00
    2        3         4        2021-01-05 10:00:00
    2        1         4        2021-03-02 10:00:00

    Returns:
    user_id num_ratings_last_30d date
    1       1                    2021-01-02
    1       2                    2021-01-21
    1       1                    2021-02-01
    1       2                    2021-02-06
    1       3                    2021-02-11
    1       2                    2021-02-20
    1       1                    2021-03-08
    1       0                    2021-03-13
    2       1                    2021-01-06
    2       0                    2021-02-05
    2       1                    2021-03-03
    2       0                    2021-04-02
    """

    df = df.sort_values(by=["user_id", "datetime"])
    grouped = df.groupby("user_id")

    def rolling_count(group: DataFrameGroupBy) -> pd.DataFrame:
        # Add a row at the end to ensure the aggregation decreases to 0
        group.loc[len(group.index)] = [
            None,
            None,
            None,
            group["datetime"].max() + pd.Timedelta(days=30),
        ]
        daily = (
            group.resample("D", origin="start_day", label="right", on="datetime")[
                ["rating"]
            ]
            .count()
            .rolling("30D")
            .sum()
            .astype(int)
        )
        return daily[daily["rating"] != daily["rating"].shift(1)]

    return (
        grouped.apply(rolling_count)
        .reset_index()
        .rename(columns={"datetime": "date", "rating": "num_ratings_last_30d"})[
            ["user_id", "num_ratings_last_30d", "date"]
        ]
    )

In [9]:
df = ratings.merge(movie_features, on="movie_id")
df.sort_values(by=["user_id", "datetime"], inplace=True)

In [ ]:
genres = [
    "action",
    "adventure",
    "animation",
    "children",
    "comedy",
    "crime",
    "documentary",
    "drama",
    "fantasy",
    "film_noir",
    "horror",
    "imax",
    "musical",
    "mystery",
    "romance",
    "sci_fi",
    "thriller",
    "war",
    "western",
]

g = df.groupby("user_id")


def user_genre_fractions(df: pd.DataFrame) -> pd.DataFrame:
    """Calculates the fraction of each genre a user has watched at each point in time.

    Example:
    user_id  movie_id  rating   datetime             action comedy ...
    1        1         5        2021-01-01 10:00:00  1      0
    1        2         4        2021-01-20 13:00:00  0      1
    1        3         3        2021-02-05 15:00:00  1      0
    1        4         2        2021-02-10 09:00:00  0      0
    2        3         4        2021-01-05 10:00:00  0      1
    2        1         4        2021-03-02 10:00:00  1      0

    Returns:
    user_id  datetime             frac_action   frac_comedy ...
    1        2021-01-01 10:00:00  0.0           0.0
    1        2021-01-20 13:00:00  1.0           0.0
    1        2021-02-05 15:00:00  0.5           0.5
    1        2021-02-10 09:00:00  0.67          0.33
    2        2021-01-05 10:00:00  0.0           0.0
    2        2021-03-02 10:00:00  0.0           1.0
    """
    g = df.groupby("user_id")
    frac = (
        g[genres].cumsum().shift(1).fillna(0).div(range(0, len(df)), axis=0).fillna(0)
    )
    frac.columns = [f"frac_{col}" for col in frac.columns]
    return pd.concat([df[["user_id", "datetime"]], frac], axis=1)


def user_genre_avg_ratings(df: pd.DataFrame) -> pd.DataFrame:
    """Calculates the average rating a user has given to each genre at each point in time.

    Has 3 as initial value for each genre.

    Example:
    user_id  movie_id  rating   datetime             action comedy ...
    1        1         5        2021-01-01 10:00:00  1      0
    1        2         4        2021-01-20 13:00:00  0      1
    1        3         3        2021-02-05 15:00:00  1      0
    1        4         2        2021-02-10 09:00:00  0      0
    2        3         4        2021-01-05 10:00:00  0      1
    2        1         4        2021-03-02 10:00:00  1      0

    Returns:
    user_id  datetime             avg_rating_action avg_rating_comedy ...
    1        2021-01-01 10:00:00  3.0               3.0
    1        2021-01-20 13:00:00  5.0               3.0
    1        2021-02-05 15:00:00  5.0               4.0
    1        2021-02-10 09:00:00  4.0               4.0
    2        2021-01-05 10:00:00  3.0               3.0
    2        2021-03-02 10:00:00  3.0               4.0
    """

    

    # Use rating value as weight
    genre_ratings = df.groupby("user_id").apply(
        lambda x: x[genres].multiply(x["rating"], axis=0)).copy().reset_index(level=0)
    

    avg = genre_ratings.groupby("user_id").cumsum().div(df.groupby("user_id")[genres].cumsum(), axis=0).shift(1).fillna(3)
    avg.columns = [f"avg_rating_{col}" for col in avg.columns]

    return pd.concat([df[["user_id", "datetime"]], avg], axis=1)

In [11]:
user_genre_avg_ratings(df).head()

/tmp/ipykernel_30489/2715871284.py:82: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  genre_ratings = df.groupby("user_id").apply(


,user_id,datetime,avg_rating_action,avg_rating_adventure,avg_rating_animation,avg_rating_children,avg_rating_comedy,avg_rating_crime,avg_rating_documentary,avg_rating_drama,...,avg_rating_film_noir,avg_rating_horror,avg_rating_imax,avg_rating_musical,avg_rating_mystery,avg_rating_romance,avg_rating_sci_fi,avg_rating_thriller,avg_rating_war,avg_rating_western
14,23474,2021-05-27 18:12:39,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
35,23474,2021-05-27 18:13:24,5.0,3.0,3.0,3.0,3.0,5.0,3.0,5.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,5.0,3.0,3.0
92,23474,2021-05-27 18:13:50,5.0,5.0,5.0,3.0,3.0,5.0,3.0,5.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,5.0,3.0,3.0
13,23474,2021-05-27 18:13:52,5.0,5.0,5.0,3.0,3.0,5.0,3.0,5.0,...,3.0,3.0,5.0,3.0,3.0,3.0,3.0,5.0,3.0,3.0
112,23474,2021-05-27 18:14:12,5.0,5.0,5.0,3.0,3.0,5.0,3.0,5.0,...,3.0,3.0,5.0,3.0,3.0,3.0,5.0,5.0,3.0,3.0


In [12]:
user_genre_fractions(df).head()

,user_id,datetime,frac_action,frac_adventure,frac_animation,frac_children,frac_comedy,frac_crime,frac_documentary,frac_drama,...,frac_film_noir,frac_horror,frac_imax,frac_musical,frac_mystery,frac_romance,frac_sci_fi,frac_thriller,frac_war,frac_western
14,23474,2021-05-27 18:12:39,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.000000,0.0,0.0
35,23474,2021-05-27 18:13:24,1.000000,0.000000,0.000000,0.0,0.0,1.000000,0.0,1.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,1.000000,0.0,0.0
92,23474,2021-05-27 18:13:50,0.500000,0.500000,0.500000,0.0,0.0,0.500000,0.0,0.500000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.500000,0.0,0.0
13,23474,2021-05-27 18:13:52,0.666667,0.333333,0.333333,0.0,0.0,0.666667,0.0,0.666667,...,0.0,0.0,0.333333,0.0,0.0,0.0,0.00,0.333333,0.0,0.0
112,23474,2021-05-27 18:14:12,0.750000,0.250000,0.250000,0.0,0.0,0.500000,0.0,0.500000,...,0.0,0.0,0.250000,0.0,0.0,0.0,0.25,0.500000,0.0,0.0
